<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R

Here we run the baseline Kabsch algorithm to best align two sets of
vectors, where there is noise on the last two vector measurements of
the ``b`` set:


In [ ]:
a = [[0, 1, 0], [0, 1, 1], [0, 1, 1]]
b = [[1, 0, 0], [1, 1.1, 0], [1, 0.9, 0]]
rot, rssd, sens = R.align_vectors(a, b, return_sensitivity=True)
rot.as_matrix()

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

When we apply the rotation to ``b``, we get vectors close to ``a``:


In [ ]:
rot.apply(b)

array([[0. , 1. , 0. ],
       [0. , 1. , 1.1],
       [0. , 1. , 0.9]])

The error for the first vector is 0, and for the last two the error is
magnitude 0.1. The `rssd` is the square root of the sum of the
weighted squared errors, and the default weights are all 1, so in this
case the `rssd` is calculated as
``sqrt(1 * 0**2 + 1 * 0.1**2 + 1 * (-0.1)**2) = 0.141421356237308``


In [ ]:
a - rot.apply(b)

array([[ 0., 0.,  0. ],
       [ 0., 0., -0.1],
       [ 0., 0.,  0.1]])

In [ ]:
np.sqrt(np.sum(np.ones(3) @ (a - rot.apply(b))**2))

0.141421356237308

In [ ]:
rssd

0.141421356237308

The sensitivity matrix for this example is as follows:


In [ ]:
sens

array([[0.2, 0. , 0.],
       [0. , 1.5, 1.],
       [0. , 1. , 1.]])

Special case 1: Find a minimum rotation between single vectors:


In [ ]:
a = [1, 0, 0]
b = [0, 1, 0]
rot, _ = R.align_vectors(a, b)
rot.as_matrix()

array([[0., 1., 0.],
       [-1., 0., 0.],
       [0., 0., 1.]])

In [ ]:
rot.apply(b)

array([1., 0., 0.])

Special case 2: One infinite weight. Here we find a rotation between
primary and secondary vectors that can align exactly:


In [ ]:
a = [[0, 1, 0], [0, 1, 1]]
b = [[1, 0, 0], [1, 1, 0]]
rot, _ = R.align_vectors(a, b, weights=[np.inf, 1])
rot.as_matrix()

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [ ]:
rot.apply(b)

array([[0., 1., 0.],
       [0., 1., 1.]])

Here the secondary vectors must be best-fit:


In [ ]:
a = [[0, 1, 0], [0, 1, 1]]
b = [[1, 0, 0], [1, 2, 0]]
rot, _ = R.align_vectors(a, b, weights=[np.inf, 1])
rot.as_matrix()

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [ ]:
rot.apply(b)

array([[0., 1., 0.],
       [0., 1., 2.]])